# Project BBT045
Question:
> How pharmaceutical pollution affect microbial community composition

In [7]:
#Load modules
#??

# Temporary

In [2]:
# Directories
# groupDirectory = /cephyr/NOBACKUP/groups/bbt045_2024/project_group1/
# resultsDirectory = /cephyr/NOBACKUP/groups/bbt045_2024/project_group1/results/
# dataDirectory = /cephyr/NOBACKUP/groups/bbt045_2024/PROJECT_DATA


# Checking stuff, remove later

##Check that we have access to the files in the common directory:
#with open("/cephyr/NOBACKUP/groups/bbt045_2024/project_group1/filelist.txt") as f: # The with keyword automatically closes the file when you are done
#    print(f.read())
#    #ref https://stackoverflow.com/questions/18256363/how-do-i-print-the-content-of-a-txt-file-in-python

#Search for packages in the softwarelist
! grep "meta" < /cephyr/NOBACKUP/groups/bbt045_2024/ProjectSoftware/bbt045-projects.yml

# Display the files in the data directory
#!ls {dataDirectory}

# Check variable directory names
#!head {dataDirectory}/change_filenames_1.sh

  - importlib-metadata=7.0.1=py310h06a4308_0
  - importlib_metadata=7.0.1=hd3eb1b0_0
  - metaphlan=4.0.6=pyhca03a8a_0


In [6]:
%%bash
DATADIR=/cephyr/NOBACKUP/groups/bbt045_2024/PROJECT_DATA;

# Test av loop och filsökning med *:
for file in `ls $DATADIR/*_1.fastq.gz  | grep -v "_lake_" | sed "s/_1.fastq.gz//"`
do
    echo $file
    #outputName=`echo $file | sed "s/\/cephyr\/NOBACKUP\/groups\/bbt045_2024\/PROJECT_DATA\///"`
    #echo $outputName
done

/cephyr/NOBACKUP/groups/bbt045_2024/PROJECT_DATA/downstream_1.5km_2012
/cephyr/NOBACKUP/groups/bbt045_2024/PROJECT_DATA/downstream_17.5km_2011
/cephyr/NOBACKUP/groups/bbt045_2024/PROJECT_DATA/downstream_2.3km_2011
/cephyr/NOBACKUP/groups/bbt045_2024/PROJECT_DATA/downstream_2.7km_2011
/cephyr/NOBACKUP/groups/bbt045_2024/PROJECT_DATA/effluent_point_2011
/cephyr/NOBACKUP/groups/bbt045_2024/PROJECT_DATA/upstream_1.9km_2011
/cephyr/NOBACKUP/groups/bbt045_2024/PROJECT_DATA/upstream_150m_2012
/cephyr/NOBACKUP/groups/bbt045_2024/PROJECT_DATA/upstream_2.2km_2011


# TO DO
- Text hur vi valde saker
- Text hur vi valde inställningar
- Referenser: manualer?
- Länk till script-filer? Fungerar det i Jupyter?
- Uppdatera text med hur scriptet körs?
- Ta bort onödiga kommentarer i kodrutorna
- Lägg alla batchfiler i en mapp?
- Renskriv batchfilernas kommentarer
- Kopiera så att det finns två batchfiler för trim? 2012 vs 2011, se kommentarer


# Quality control and trimming

## FastQC
Do FastQC for all samples except those with "lake" in their name. 

In [9]:
%%script false --no-raise-error
for file in `ls /cephyr/NOBACKUP/groups/bbt045_2024/PROJECT_DATA/*.fastq.gz | grep -v "_lake_"`
do
    fastqc $file -o /cephyr/NOBACKUP/groups/bbt045_2024/project_group1/results/fastqc
done

#Tekniskt sett är det detta som körs:
for file in `ls *.fastq.gz | grep -v "_lake_"`
do
    apptainer exec $CONTAINER_LOC fastqc $file -o $WORKING_TMP
done

## Bash script for trimmomatic

The following sbatch script is to run trimmomatic on all samples except those with "_lake_" in their name

NOTE:
- Behövde använda -phred64 på 2011-filerna, gav error på dem annars när jag körde den. När ska köra 2012-filerna: byt ut år i `cp`och `-phred64` till `-phred33`

Paired End Mode:
ref: http://www.usadellab.org/cms/?page=trimmomatic
> trimmomatic PE [-threads <threads>] [-phred33 | -phred64] [-trimlog <logFile>] <input 1> <input 2> <paired output 1> <unpaired output 1> <paired output 2> <unpaired output 2> <step 1> ...

In [3]:
%%script false --no-raise-error
# Extra för min förståelse, ta bort?
WORKDIR=/cephyr/NOBACKUP/groups/bbt045_2024/project_group1/results/trimmomatic;
WORKING_TMP=$TMPDIR/TRIM_TMP;

mkdir $WORKING_TMP;
cd $WORKING_TMP;

cp /cephyr/NOBACKUP/groups/bbt045_2024/PROJECT_DATA/*2011*.fastq.gz $WORKING_TMP

### Running Trimmomatic
for file in `ls *_1.fastq.gz  | grep -v "_lake_" | sed "s/_1.fastq.gz//"`
do
    apptainer exec $CONTAINER_LOC trimmomatic PE -phred64 \
                    $file\_1.fastq.gz $file\_2.fastq.gz \
                    $file\_1.trimmed.fastq.gz $file\_1.un.trimmed.fastq.gz \
                    $file\_2.trimmed.fastq.gz $file\_2.un.trimmed.fastq.gz \
                    TOPHRED33 \
                    LEADING:30 TRAILING:30 \
                    MINLEN:60
done


# Extra, ta bort:
    #question: does this output the files in $WORKING_TMP? Or do we need to add this before the output file names?
            # or do we need to remove everything but the name and use that instead of $file as below?
    #question/look at output: autodetect phred score? (run one of each and see? 33 first)
    #question Paria/Alva: vilket score ska vi använda?
    #question: Need to remove adapters? Nearly no present?
    #question: trimmomatic phred score? TOPHRED64 if already PHRED64??

    #ls $file\_1.fastq.gz $file\_2.fastq.gz
    #apptainer exec $CONTAINER_LOC trim_galore --quality 20 --length 60 --cores 8 --paired -o /cephyr/NOBACKUP/groups/bbt045_2024/projectAliceRita5 $i\_1.fastq.gz $i\_2.fastq.gz

# Johan brukar altid trimma adapters innan uppladdning, kanske därför vi inte ser några i fastqc-output

# FastQC (again)
Körde fastqc igen, fast denna gång på alla filer i project_group1/results/trimmomatic/

In [10]:
%%script false --no-raise-error
for file in `ls /cephyr/NOBACKUP/groups/bbt045_2024/PROJECT_DATA/*.fastq.gz | grep -v "_lake_"`
do
    fastqc $file -o /cephyr/NOBACKUP/groups/bbt045_2024/project_group1/results/fastqc
done

#Tekniskt sett är det detta som körs:
for file in `ls *.fastq.gz | grep -v "_lake_"`
do
    apptainer exec $CONTAINER_LOC fastqc $file -o $WORKING_TMP
done

# Metaphlan4

Running metaphlan4

Wiki: https://github.com/biobakery/biobakery/wiki/metaphlan4 

In [7]:
%%script false --no-raise-error
for file in `ls *.fastq.gz | grep -v ".un.trimmed.fastq.gz"| sed "s/_1.trimmed.fastq.gz//"`
do
    apptainer exec $CONTAINER_LOC metaphlan $file\_1.trimmed.fastq.gz,$file\_2.trimmed.fastq.gz \
                                            --input_type fastq --nproc 4 \
                                            
done
for i in SRS*.fasta.gz; do metaphlan $i --input_type fasta --nproc 4 > ${i%.fasta.gz}_profile.txt; done

#todo Lägga till: ?
# --unclassified_estimation (kanske roligt att veta?)
# --add_viruses (eller inte eftersom det är microbial community, --> ≠ virus?)
# Kontrollera size av --bowtie2out $file.bowtie2.bz2 efter första testet, giganorm fil?

## Visualizing metaphlan output

In [8]:
%%script false --no-raise-error
# Merge the _profile tables for each sample into a single file
merge_metaphlan_tables.py *_profile.txt > merged_abundance_table.txt
#ref: https://github.com/biobakery/biobakery/wiki/metaphlan4#16-merging-metaphlan-profiles

# Isolate the species-resolved rows and then remove clutter 
grep -E "s__|SRS" merged_abundance_table.txt \
    grep -v "t__" \
    sed "s/^.*|//g" \
    sed "s/SRS[0-9]*-//g" \
    > merged_abundance_table_species.txt
#ref https://github.com/biobakery/biobakery/wiki/metaphlan4#211-generate-a-species-only-abundance-table

#todo förbättra kommentarer ovan, se ref:s för mer info